### Q5 (h) 

In [1]:
import time
import numpy as np
from scipy import sparse
from scipy.sparse import spdiags
from scipy.sparse import kron
from scipy.sparse import eye
from scipy.sparse import linalg
from scipy.sparse.linalg import eigs

In [2]:
def lcd(beta, gamma, N):
    ee = np.ones((1,N))
    a = 5
    b = -1 - gamma
    c = -1 - beta
    d = -1 + beta
    e = -1 + gamma
    t1 = sparse.spdiags(np.vstack([c*ee, a*ee, d*ee]), np.arange(-1,2), N, N)
    t2 = sparse.spdiags(np.vstack([b*ee, np.zeros((1,N)), e*ee]), np.arange(-1,2), N, N)
    matrix = sparse.kron(sparse.eye(N,N),t1) + sparse.kron(t2,sparse.eye(N,N))
    return matrix

In [3]:
tol = 1e-12 # tolerance of errors
A = lcd(0,0,50)
vals, vecs = np.linalg.eig(A.toarray())
alpha = 2/(max(vals) + min(vals))
beta = 1 - (2*max(vals)/(max(vals)-min(vals)))

In [4]:
# stationary version
x = np.zeros((2500,1))
b = np.ones((2500,1))
e_norm1 = np.linalg.norm(b - A*x)
k = 0
while e_norm1 >= tol and k < 20000:
    r = b - A*x
    x = x + alpha * r
    e_norm1 = np.linalg.norm(r)
    k = k + 1

In [5]:
# acceleration
x0 = np.zeros((2500,1))
b = np.ones((2500,1))
e_norm2 = np.linalg.norm(b - A*x0)
x1 = x0 + alpha * e_norm2
w = 2
n = 0
while e_norm2 >= tol and n < 20000:
    r = b - A*x1
    w = (1 - (w/(4*beta**2)))**(-1)
    x2 = x0 + alpha * w * r + w*(x1-x0)
    x0 = x1
    x1 = x2
    e_norm2 = np.linalg.norm(r)
    n = n + 1

In [6]:
print(e_norm1)
print(k)
print(e_norm2)
print(n)

8.683687935579432e-13
141
6.653483407327103e-13
56


Implementing the accelerated variant takes much fewer iterations than the original stationary version. 